In [24]:
# Load MNIST data 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [25]:
import tensorflow as tf 
sess = tf.InteractiveSession()

C:\Users\ychen\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [26]:
# Softmax Regression model 
# Nodes for input images and targe output classes 
# Used and modified by computation graph 
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
# Weight and bias
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# Initialize variables 
sess.run(tf.initialize_all_variables())

In [27]:
# Predicted Class and Loss Function 
y = tf.nn.softmax(tf.matmul(x,W) + b)
# Cross entropy loss 
cross_entropy = tf.reduce_mean( \
        -tf.reduce_sum(y_*tf.log(y), reduction_indices=[1]))
# reduce_sum: sum across classes
# reduce_mean: average over sums 

In [28]:
# Training 
# Steepest gradient descent, step length 0.5, cross entropy
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# Adding operations to computation graph 
# Compute gradients, compute parameter update step, apply update step to parameters

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
# Load 100 training examples each iteration 
# Run train_step, using feed_dict to replace placeholders 

In [29]:
# Evaluate the model 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean( \
    tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9191


In [30]:
# Convolutional Network 
# Weight initialization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Convolution and Pooling 
def conv2d(x, W):
    return tf.nn.conv2d(x, \
        W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], \
        strides=[1, 2, 2, 1], padding='SAME')

In [32]:
# First Convolutional Layer 
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [33]:
# Second Convolutional Layer 
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [34]:
# Densely Connected Layer 
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [36]:
# Dropout 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [37]:
# Readout Layer 
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [39]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12
step 100, training accuracy 0.8
step 200, training accuracy 0.9
step 300, training accuracy 0.92
step 400, training accuracy 0.88
step 500, training accuracy 0.92
step 600, training accuracy 0.96
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.98
test accuracy 0.9608
